# Bassed on Captum Interpret regression models

    In 2020 Stackoverflow survey Darp programming language have popularity of 4% between 57,378 responses, but flutter framework, which is Dart baset is far more popular with 7.3% from 40,314 responses. Percentage is simmilar for professional developers only. 
    Let us investigate!


In [5]:
import numpy as np

from os import path


import matplotlib.pyplot as plt

#scikit-learn related imports
import sklearn
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# pytorch relates imports
import torch
import torch.nn as nn
import torch.optim as optim

# imports from captum library
from captum.attr import LayerConductance, LayerActivation, LayerIntegratedGradients
from captum.attr import IntegratedGradients, DeepLift, GradientShap, NoiseTunnel, FeatureAblation

# additions to original imports
import pandas as pd

In [6]:
## Data loading and pre-processing

Let's load boston house prices dataset and corresponding labels from scikit-learn libra

In [12]:
df01 = pd.read_csv('survey_results_public.csv')

In [13]:
df01.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


In [18]:
filter_df = df01[df01["LanguageWorkedWith"].str.contains("Dart",na = False)]

In [19]:
filter_df

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
42,43,I am a developer by profession,Yes,NaN,16,Yearly,135000.0,135000.0,United States,United States dollar,...,Easy,Appropriate in length,NaN,"Computer science, computer engineering, or sof...",NaN,ASP.NET,Just as welcome now as I felt last year,45.0,36,29
52,53,I am a developer by profession,Yes,17.0,9,NaN,NaN,NaN,United States,United States dollar,...,Easy,Appropriate in length,No,NaN,NaN,Flask,A lot less welcome now than last year,5.0,8,3
130,132,I am a developer by profession,No,38.0,13,Monthly,1200.0,15564.0,Spain,European Euro,...,Neither easy nor difficult,Appropriate in length,No,NaN,NaN,Gatsby;React.js,A lot more welcome now than last year,20.0,5,4
131,133,I am a developer by profession,Yes,NaN,18,Yearly,45000.0,24864.0,Bulgaria,Bulgarian lev,...,Easy,Appropriate in length,No,"Computer science, computer engineering, or sof...",Angular;ASP.NET Core,Angular;Angular.js;ASP.NET;ASP.NET Core;Expres...,A lot more welcome now than last year,40.0,8,5
184,186,I am a developer by profession,Yes,31.0,25,Monthly,2200.0,28536.0,Slovenia,European Euro,...,Easy,Appropriate in length,No,"Computer science, computer engineering, or sof...",NaN,Express,Just as welcome now as I felt last year,60.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64401,56163,NaN,Yes,NaN,Younger than 5 years,NaN,NaN,NaN,Poland,NaN,...,Easy,Too short,No,NaN,Angular;Angular.js;ASP.NET;ASP.NET Core;Django...,Angular;Angular.js;ASP.NET;ASP.NET Core;Django...,Just as welcome now as I felt last year,NaN,Less than 1 year,NaN
64413,58498,NaN,Yes,32.0,23,NaN,NaN,NaN,United States,NaN,...,Neither easy nor difficult,Appropriate in length,No,Mathematics or statistics,ASP.NET;ASP.NET Core;Flask;Gatsby;jQuery;React...,ASP.NET;ASP.NET Core;Flask;Gatsby;jQuery;React...,NaN,NaN,8,Less than 1 year
64426,60134,NaN,Yes,22.0,10,NaN,NaN,NaN,Spain,NaN,...,Easy,Appropriate in length,No,"Computer science, computer engineering, or sof...",Express;React.js,Express;React.js;Spring,Just as welcome now as I felt last year,NaN,10,NaN
64429,61031,NaN,Yes,NaN,23,NaN,NaN,NaN,Spain,NaN,...,Easy,Appropriate in length,No,NaN,Angular;Angular.js;React.js;Vue.js,Angular;Angular.js,Just as welcome now as I felt last year,NaN,1,1


In [20]:
filter_df2 = df01[df01["LanguageWorkedWith"].str.contains("Dart",na = False) or df01["MiscTechWorkedWith"].str.contains("Flutter",na = False)]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().